In [ ]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
q, k, v = inputs, inputs, inputs

attn_scores = q @ k.T

# normalize with softmax
attn_weight = torch.softmax(attn_scores, dim=-1)
display(attn_scores.shape)
display(attn_weight.shape)
display(attn_weight.sum(dim=-1)) # check if sum to 1

all_context = attn_weight @ v
display(all_context.shape)
print(all_context)



torch.Size([6, 6])

torch.Size([6, 6])

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

torch.Size([6, 3])

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [ ]:
import math
torch.manual_seed(123)
d_in = 3
d_out = 2
W_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

q, k, v = inputs @ W_q, inputs @ W_k, inputs @ W_v
d_k = k.shape[-1]
attn_scores = q @ k.T
attn_weight = torch.softmax(attn_scores / math.sqrt(d_k), dim=-1)
z = attn_weight @ v
print(z)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


In [ ]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_q = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.W_k = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        self.W_v = nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
        
    def forward(self, x):
        k = x @self.W_k
        q = x @self.W_q
        v = x @self.W_v
        d_k = k.shape[-1]
        attn_scores = q @ k.T
        attn_weight = torch.softmax(attn_scores / math.sqrt(d_k), dim=-1)
        z = attn_weight @ v
        return z


In [ ]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in=3, d_out=2)
z_v1 = sa_v1(inputs)
print(z_v1)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [ ]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)
        d_k = k.shape[-1]
        scores = q @ k.T
        weights = torch.softmax(scores / math.sqrt(d_k), dim=-1)
        z = weights @ v
        return z

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in=3, d_out=2)
z_v2 = sa_v2(inputs)
print(z_v2)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [ ]:
mask = torch.tril(torch.ones(attn_weight.shape))
masked_attn_weight = attn_weight*mask
normalized_masked_attn_weight = masked_attn_weight / masked_attn_weight.sum(dim=-1, keepdim=True)
print(normalized_masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3986, 0.6014, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2526, 0.3791, 0.3683, 0.0000, 0.0000, 0.0000],
        [0.2265, 0.2839, 0.2794, 0.2103, 0.0000, 0.0000],
        [0.1952, 0.2363, 0.2331, 0.1820, 0.1534, 0.0000],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])


In [ ]:
import torch.nn as nn
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
    def forward(self, x):
        batch, num_tokens, d_in = x.shape
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)
        d_k = k.shape[-1]
        scores = q @ k.transpose(-2, -1)
        scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], float('-inf'))
        weights = torch.softmax(scores / math.sqrt(d_k), dim=-1)
        weights = self.dropout(weights)
        z = weights @ v
        return z
    

In [ ]:
torch.manual_seed(123)
batch = torch.stack((inputs, inputs), dim=0)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.1, num_heads=1, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out // num_heads, context_length, dropout, qkv_bias) for _ in range(num_heads)
        ])
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in=3, d_out=4, context_length=context_length, num_heads=2, dropout=0.0)
z_mha = mha(batch)
print(z_mha)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


In [ ]:
# tmp_data = torch.arange(0, 2*3*5*4)
# tmp_k = tmp_data.view(2, 3, 5*4)
# print(tmp_k.shape)
# tmp_q = tmp_k
# tmp_v = tmp_k
# tmp_k = tmp_k.view(2,3,5,4)
# print(tmp_k.shape)
# tmp_q = tmp_q.view(2,3,5,4)
# tmp_v = tmp_v.view(2,3,5,4)
# # print(tmp_q.shape)
# # print(tmp_v.shape)
# tmp_k = tmp_k.transpose(1,2)
# print(tmp_k.shape)
# tmp_q = tmp_q.transpose(1,2)
# tmp_v = tmp_v.transpose(1,2)
# print("tmp_q shape after transpose:", tmp_q.shape)
# # print(tmp_v.shape)
# tmp_attn_scores = tmp_q @ tmp_k.transpose(-1, -2)
# print(tmp_attn_scores.shape)

torch.triu(torch.ones(4,4), diagonal=1)

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.1, num_heads=1, qkv_bias=False):
        assert (d_out % num_heads == 0), "d_out must be divisiable by num_heads"
        super().__init__()
        self.d_in = d_in
        self.num_heads = num_heads
        self.d_head = d_out // num_heads
        self.w_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
    def forward(self, x):
        import math
        
        q = self.w_q(x)
        k = self.w_k(x)
        v = self.w_v(x)
        
        b, context_length, d_out = q.shape
        split_view = lambda x: x.view(b, context_length, self.num_heads, self.d_head)\
            .transpose(1, 2)
        q = split_view(q)
        k = split_view(k)
        v = split_view(v)
        
        attn_scores = q @ k.transpose(-1, -2) / math.sqrt(k.shape[-1])
        attn_scores.masked_fill_(self.mask.bool(), -torch.inf)
        weight = torch.softmax(attn_scores, dim=-1)
        weight = self.dropout(weight)
        
        z = weight @ v
        z = z.transpose(1, 2).contiguous().view(b, context_length, d_out)
        z = self.out_proj(z)
        return z
        

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttention(d_in=3, d_out=4, context_length=context_length, num_heads=2, dropout=0.0)
z_mha = mha(batch)
print(z_mha)

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)


In [ ]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]],

        [[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
